# Simple Census Grab

### Load a few packages.

In [ ]:
import os

import pandas as pd
import geopandas as gpd
import mplleaflet

from census import Census
from us import states

In [ ]:
c = Census("f31f8863c1a1b23699154a77add863aeef43a896", year = 2014)

### Get poverty data from the census.

In [ ]:
il_poverty = c.acs5.state_county_tract(('NAME', 'B17020_001E', 'B17020_002E'), 
                                       states.IL.fips, '*', Census.ALL)

pov_df = pd.io.json.json_normalize(il_poverty)

pov_df['rate'] = 100 * pov_df['B17020_002E'].astype(int) / pov_df['B17020_001E'].astype(int)

### Get shapefiles of census tracts.

In [ ]:
# os.system("wget http://www2.census.gov/geo/tiger/GENZ2014/shp/cb_2014_17_tract_500k.zip")
# os.system("unzip cb_2014_17_tract_500k.zip")
tract_df = gpd.read_file("cb_2014_17_tract_500k.shp")

### Merge them together.

In [ ]:
merged_df = tract_df.merge(pov_df, left_on  = ["STATEFP", "COUNTYFP", "TRACTCE"], 
                                   right_on = ["state", "county", "tract"])

In [ ]:
with open('census.geojson', 'w') as f:
    f.write(merged_df.to_json())

### See it.

In [ ]:
import folium, numpy as np

colormap = folium.colormap.linear.PuOr.scale(0, 50)

m = folium.Map([41.7943,-87.5907], zoom_start=13, tiles='cartodbpositron', 
               max_zoom=14, min_zoom=6, attr = "")

folium.GeoJson(merged_df,
               style_function=lambda feature: {
                'fillColor': colormap(feature['properties']['rate']) if feature['properties']['rate'] else 'k',
                'color' : 'k', 'weight' : 0.3,
                'fillOpacity' : 0.4 if feature['properties']['rate'] else 0,
               }).add_to(m)

colormap.caption = 'Percent Poverty'
colormap.add_to(m)

m